# Fragestellung 1.4: Wie hat sich der Anteil fossiler Heizungen in den letzten 20 Jahren in einer Gemeinde verändert? 

Um diese Fragen zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. Die Daten werden nach Gemeinde, Bauperiode und Energiequelle gruppiert. Anschliessend werden die einzelnen Objekte gezählt. Anschliessend wird die resultierende Liste als CSV-Datei exportiert.

## Notebook vorbereiten und Daten importieren

In einem ersten Schritt werden die notwendigen Python Bibliotheken importiert und die Daten als Dataframe reingeladen.

In [1]:
# Zu verwendende Bibliotheken importieren
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [2]:
# Datenquelle definieren
url = urlopen("https://public.madd.bfs.admin.ch/ch.zip")

# ZIP-File herunterladen und Dataframe erstellen
zipfile = ZipFile(BytesIO(url.read()))
df = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'), 
                     sep='\t')

df.head()

/tmp/ipykernel_32135/2263257821.py:6: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zipfile.open('gebaeude_batiment_edificio.csv'),


,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GWAERDATH2,GWAERZW1,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT
0,11513432,ZH,1,Aeugst am Albis,CH540120777857,0.0,1573.0,NaN,NaN,1199,...,NaN,7630.0,7530.0,860.0,2002-09-05,NaN,NaN,NaN,NaN,2022-11-14
1,11513433,ZH,1,Aeugst am Albis,CH587820017717,0.0,1543.0,NaN,NaN,1198,...,NaN,7650.0,7560.0,860.0,2002-09-05,NaN,NaN,NaN,NaN,2022-11-14
2,11517090,ZH,1,Aeugst am Albis,CH707701872012,0.0,1361.0,NaN,NaN,1017,...,NaN,7630.0,7530.0,860.0,2002-09-05,NaN,NaN,NaN,NaN,2022-11-14
3,1600000,ZH,1,Aeugst am Albis,CH487820017751,0.0,1631.0,NaN,NaN,164,...,NaN,7630.0,7530.0,869.0,2001-11-29,NaN,NaN,NaN,NaN,2022-11-14
4,1600001,ZH,1,Aeugst am Albis,CH567720017811,0.0,1579.0,NaN,NaN,154,...,NaN,7630.0,7530.0,860.0,2001-11-29,NaN,NaN,NaN,NaN,2022-11-14


## Daten vorbereiten

Als nächstes werden die Daten vorprozessiert. Dabei werden die Attributnamen geändert, unwesentliche Spalten gelöscht, Werte klassiert und NULL Werte behandelt.
Nach diesem Schritt sind die Daten bereinigt und können für die Analyse verwendet werden.

In [3]:
# Die gewünschten Attribute sinnvoll umbenennen und in ein neues Dateframe speichern.
buildings = df.rename(columns={"GDEKT":"Kanton",
                               "GGDENAME": "Gemeinde",
                               "GKODE": "lat",
                               "GKODN": "lon",
                               "GKAT":"Gebaeudekategorie",
                               "GBAUJ":"Baujahr",
                               "GBAUP":"Bauperiode",
                               "GANZWHG":"AnzahlWohnungen",
                               "GEBF":"Energiebezugsflaeche",
                               "GWAERZH1":"Waermeerzeuger",
                               "GENH1":"Energiequelle",
                               "GWAERSCEH1":"InfoquelleHeizung"})

In [4]:
# Die nicht verwendeten Attribute löschen
buildings = buildings.drop(['GGDENR','EGRID','LGBKR','LPARZ','LPARZSX','LTYP','GEBNR','GBEZ','GKSCE','GSTAT','GKLAS','GBAUM','GABBJ','GAREA','GVOL','GVOLNORM','GVOLSCE','GASTW','GAZZI','GSCHUTZR','GWAERDATH1','GWAERZH2','GENH2','GWAERSCEH2','GWAERDATH2','GWAERZW1','GENW1','GWAERSCEW1','GWAERDATW1','GWAERZW2','GENW2','GWAERSCEW2','GWAERDATW2','GEXPDAT'], axis=1)

In [5]:
# Die korrekten Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
buildings['Energiequelle'] = buildings.Energiequelle.replace({
                                        7500: 'Keine',
                                        7501: 'Weitere',
                                        7510: 'Weitere',
                                        7511: 'Weitere',
                                        7512: 'Weitere',
                                        7513: 'Weitere',
                                        7520: 'Gas',
                                        7530: 'Heizöl',
                                        7540: 'Weitere',
                                        7541: 'Weitere',
                                        7542: 'Weitere',
                                        7543: 'Weitere',
                                        7550: 'Weitere',
                                        7560: 'Elektrizität',
                                        7570: 'Weitere',
                                        7580: 'Weitere',
                                        7581: 'Weitere',
                                        7582: 'Weitere',
                                        7598: 'Unbestimmt',
                                        7599: 'Weitere'
                                            })


In [6]:
# Fehlende Werte werden wie folgt berücksichtigt.

# Fehlende Bauperiode wird zu "Vor 1919"
buildings['Bauperiode'] = buildings['Bauperiode'].fillna(8011)

# Fehlende Energiequelle wird zu "Unbestimmt"
buildings['Energiequelle'] = buildings['Energiequelle'].fillna('Unbestimmt')

In [7]:
# Bereinigtes DataFrame anschauen
buildings

,EGID,Kanton,Gemeinde,lat,lon,Gebaeudekategorie,Baujahr,Bauperiode,AnzahlWohnungen,Energiebezugsflaeche,Waermeerzeuger,Energiequelle,InfoquelleHeizung
0,11513432,ZH,Aeugst am Albis,2679647.268,1237500.347,1020.0,2000.0,8019.0,2.0,NaN,7430.0,Heizöl,860.0
1,11513433,ZH,Aeugst am Albis,2680635.900,1236936.229,1020.0,2000.0,8019.0,1.0,NaN,7430.0,Heizöl,860.0
2,11517090,ZH,Aeugst am Albis,2679016.828,1235828.156,1020.0,1993.0,8018.0,1.0,NaN,7431.0,Heizöl,860.0
3,1600000,ZH,Aeugst am Albis,2679677.869,1237453.256,1020.0,1845.0,8011.0,3.0,NaN,7430.0,Heizöl,869.0
4,1600001,ZH,Aeugst am Albis,2679692.997,1237491.284,1030.0,1900.0,8011.0,1.0,NaN,7436.0,Heizöl,869.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069089,984473,JU,La Baroche,2582768.637,1251359.261,1020.0,NaN,8013.0,1.0,NaN,7430.0,Heizöl,860.0
3069090,984474,JU,La Baroche,2582743.072,1251406.135,1040.0,NaN,8015.0,1.0,NaN,7430.0,Heizöl,860.0
3069091,984475,JU,La Baroche,2584600.501,1251495.804,1020.0,NaN,8019.0,1.0,NaN,7436.0,Gas,860.0
3069092,984476,JU,La Baroche,2583393.948,1251448.443,1020.0,NaN,8011.0,1.0,NaN,7436.0,Weitere,860.0


## Anzahl Gebäude pro Gemeinde

In diesem Schritt wird die Anzahl Gebäude pro Gemeinde und Bauperiode ausgewertet. Die Anzahl Gebäude wird später verwendet um die prozentuale Verteilung der Energieträger zu berechnen.

In [8]:
# Zählt alle Gebäude pro Gemeinde, Bauperiode und Energiequelle
GebaeudeGemeindeTotal = buildings[['Gemeinde','Bauperiode']].value_counts().reset_index()
GebaeudeGemeindeTotal

,Gemeinde,Bauperiode,0
0,Zürich,8012.0,15939
1,Zürich,8011.0,12944
2,Zürich,8013.0,12101
3,Bern,8011.0,7670
4,Basel,8012.0,7561
...,...,...,...
27756,Treytorrens (Payerne),8021.0,1
27757,Saas-Almagell,8020.0,1
27758,Champtauroz,8016.0,1
27759,Champtauroz,8019.0,1


In [9]:
# Die Spaltennamen des DataFrames neu benennen
GebaeudeGemeindeTotal.columns  = ['Gemeinde','Bauperiode','GebaeudeTotal']
GebaeudeGemeindeTotal

,Gemeinde,Bauperiode,GebaeudeTotal
0,Zürich,8012.0,15939
1,Zürich,8011.0,12944
2,Zürich,8013.0,12101
3,Bern,8011.0,7670
4,Basel,8012.0,7561
...,...,...,...
27756,Treytorrens (Payerne),8021.0,1
27757,Saas-Almagell,8020.0,1
27758,Champtauroz,8016.0,1
27759,Champtauroz,8019.0,1


## Anzahl Gebäude pro Gemeinde, Bauperiode und Energiequelle zählen und als DataFrame speichern

In diesem Schritt werden die Anzahl Gebäude pro Gemeinde, Bauperiode und Energiequelle ausgewertet.

In [10]:
# Zählt alle Gebäude pro Gemeinde, Bauperiode und Energiequelle
Gemeindeliste = buildings[['Gemeinde','Bauperiode','Energiequelle']].value_counts().reset_index()

In [11]:
# Die Spaltennamen des DataFrames neu benennen
Gemeindeliste.columns  = ['Gemeinde','Bauperiode','Energiequelle','Anzahl']
Gemeindeliste

,Gemeinde,Bauperiode,Energiequelle,Anzahl
0,Zürich,8011.0,Gas,7008
1,Zürich,8012.0,Gas,6619
2,Zürich,8012.0,Heizöl,4233
3,Basel,8012.0,Gas,3808
4,Zürich,8013.0,Heizöl,3728
...,...,...,...,...
114336,Montricher,8012.0,Gas,1
114337,Montricher,8012.0,Keine,1
114338,Trun,8020.0,Keine,1
114339,Trun,8020.0,Heizöl,1


## Die beiden Dataframes zusammenführen

Nun werden die beiden Dataframe zusammengeführt.

In [12]:
Gemeinden = pd.merge(Gemeindeliste, GebaeudeGemeindeTotal, on=['Gemeinde','Bauperiode'])
Gemeinden

,Gemeinde,Bauperiode,Energiequelle,Anzahl,GebaeudeTotal
0,Zürich,8011.0,Gas,7008,12944
1,Zürich,8011.0,Heizöl,2098,12944
2,Zürich,8011.0,Keine,1996,12944
3,Zürich,8011.0,Weitere,1584,12944
4,Zürich,8011.0,Unbestimmt,224,12944
...,...,...,...,...,...
114336,Buseno,8021.0,Unbestimmt,1,2
114337,Buseno,8021.0,Elektrizität,1,2
114338,Busswil bei Melchnau,8016.0,Elektrizität,1,3
114339,Busswil bei Melchnau,8016.0,Weitere,1,3


## Daten als CSV-Datei exportieren

Zum Schluss wird das Dataframe als CSV-Datei exportiert. Diese Daten werden anschliessend für die Visualisierung verwendet.

In [13]:
# Als CSV-File exportieren
Gemeinden.to_csv('Daten/Gemeindeliste_1-4.csv', index = False, header=True, sep=',', encoding="utf-8-sig")